## 1. Retrieve Slice
For initial stages, using CreateSlice.ipynb from `/teaching-materials/teaching-materials/assignments/Exploring Queues`

In [1]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()
import json
import traceback
import time

slice_name = "ECE6400-Final-DDoS-Sim"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,5be8419d-0ad5-4c79-8396-258fc7e893f5
Bastion Host,bastion.fabric-testbed.net
Bastion Username,kguerns_0000197554
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
bd029200-344a-4a36-bada-fa2e73917744,Bot_0,1,2,10,default_ubuntu_20,qcow2,eduky-w7.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fedf:d037,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fedf:d037,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5866da32-5f1d-45e5-a0f8-fcf7415acbd5,Bot_1,1,2,10,default_ubuntu_20,qcow2,eduky-w6.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fedd:4b74,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fedd:4b74,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f9660c5d-5077-47c9-8af4-89b4c5f8343e,Bot_2,1,2,10,default_ubuntu_20,qcow2,eduky-w15.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe20:e1c1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe20:e1c1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d28f42b0-fd0c-41b3-9b52-7e90eb3da1d5,Bot_3,1,2,10,default_ubuntu_20,qcow2,eduky-w17.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe04:8106,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe04:8106,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1f71d876-5684-4858-a9c6-8f1a94f404b5,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fed9:c7e7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fed9:c7e7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8a7e6b35-16af-45c7-9dfe-dd96c888ee0a,Target,1,2,10,default_ubuntu_20,qcow2,eduky-w13.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fea8:bc45,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fea8:bc45,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cfcce15b-8d92-4523-90e2-20170118f94d,User,1,2,10,default_ubuntu_20,qcow2,eduky-w18.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe3d:bce1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe3d:bce1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
bd029200-344a-4a36-bada-fa2e73917744,Bot_0,1,2,10,default_ubuntu_20,qcow2,eduky-w7.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fedf:d037,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fedf:d037,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5866da32-5f1d-45e5-a0f8-fcf7415acbd5,Bot_1,1,2,10,default_ubuntu_20,qcow2,eduky-w6.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fedd:4b74,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fedd:4b74,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f9660c5d-5077-47c9-8af4-89b4c5f8343e,Bot_2,1,2,10,default_ubuntu_20,qcow2,eduky-w15.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe20:e1c1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe20:e1c1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d28f42b0-fd0c-41b3-9b52-7e90eb3da1d5,Bot_3,1,2,10,default_ubuntu_20,qcow2,eduky-w17.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe04:8106,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe04:8106,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1f71d876-5684-4858-a9c6-8f1a94f404b5,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fed9:c7e7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fed9:c7e7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8a7e6b35-16af-45c7-9dfe-dd96c888ee0a,Target,1,2,10,default_ubuntu_20,qcow2,eduky-w13.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fea8:bc45,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fea8:bc45,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cfcce15b-8d92-4523-90e2-20170118f94d,User,1,2,10,default_ubuntu_20,qcow2,eduky-w18.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe3d:bce1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe3d:bce1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


### 2. Load Nodes

In [2]:
# Setup constants
nBots = 4

# Load nodes
bots = [slice.get_node(name=f"Bot_{i}") for i in range(nBots)]
target = slice.get_node(name="Target")
controller = slice.get_node(name="Controller")
user = slice.get_node(name="User")

ip_target = "10.0.0.2"

### 3. Baseline Metric Collection

In [3]:
#print("\n===== Collect Baseline Metrics =====")
#print("\n--- Baseline Latency & Packet Loss (Target -> Controller) ---")
#print(target.execute(f"ping -i 0.5 -c 20 {controller.get_management_ip()}"))

#print("\n--- Baseline Latency & Packet Loss (User -> Target) ---")
#print(user.execute(f"ping -i 0.5 -c 20 {ip_target}"))

#print("\n--- Baseline Queue Status on Target ---")
#print(target.execute("sudo tc -s qdisc show dev enp7s0"))

#print("\n--- Baseline CPU Usage on Target ---")
#print(target.execute("top -bn1 | head -n 10"))

#print("\n--- Baseline Network Traffic Stats on Target ---")
#print(target.execute("ifconfig enp7s0"))

#time.sleep(2)


===== Collect Baseline Metrics =====

--- Baseline Latency & Packet Loss (Target -> Controller) ---
PING 2610:1e0:1700:206:f816:3eff:fed9:c7e7(2610:1e0:1700:206:f816:3eff:fed9:c7e7) 56 data bytes
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=1 ttl=64 time=2.37 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=2 ttl=64 time=0.331 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=3 ttl=64 time=0.370 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=4 ttl=64 time=0.279 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=5 ttl=64 time=0.311 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=6 ttl=64 time=0.318 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=7 ttl=64 time=0.292 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=8 ttl=64 time=0.320 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fed9:c7e7: icmp_seq=9 ttl=64 time=0.380 ms
64 bytes from 2610:1e0:1700:206:f816:3e

### 2. Sending instructions to the bots from the controller

In [37]:
import threading

# Reduce target's input rate to 1kb/s and set its queue size to 1kb
target.execute("sudo tc qdisc add dev enp7s0 root tbf rate 1kbit limit 1kb burst 10kb")

# Threading Functions
def start_ping_flood(bot, ip_target):
    try:
        bot.execute("sudo rm ping_output.txt")
        print(f"Starting ping flood attack from {bot.get_name()}...")
        bot.execute(f"nohup sudo ping -f -c 1000 {ip_target} > ping_output.txt 2>&1 &")
    except Exception as e:
        print(f"Error starting ping flood on {bot.get_name()}: {e}\n")

def start_user_ping(user, ip_target):
    try:
        user.execute("sudo rm ping_output.txt ping_pid.txt")
        print(f"Starting ping from {user.get_name()}...")
        user.execute(f"nohup sudo ping -i 0.2 -c 300 {ip_target} > ping_output.txt 2>&1 & echo $! > ping_pid.txt")
    except Exception as e:
        print(f"Error starting ping on {user.get_name()}: {e}\n")

def wait_for_ping_to_finish(bot):
    print(f"Waiting for {bot.get_name()} to finish ping flood...")
    while True:
        stdout, sterr = bot.execute("pgrep ping", quiet=True)
        if stdout.strip() == "":
            print(f"{bot.get_name()} finished.")
            break
        time.sleep(2)

def wait_for_ping_to_finish_user(bot):
    print(f"Waiting for {bot.get_name()} to finish ping flood...")
    stdout, stderr = bot.execute("cat ping_pid.txt", quiet=True)
    pid = stdout.strip()
    if pid == "":
        print(f"No PID file found on {bot.get_name()}.")
        return
    while True:
        stdout, stderr = bot.execute(f"ps -p {pid}", quiet=True)
        if "ping" not in stdout:
            print(f"{bot.get_name()} finished ping.")
            time.sleep(1)
            break
        time.sleep(2)

def collect_ping_results(bot):
    try:
        bot.download_file(f"{bot.get_name()}_ping_result.txt", "ping_output.txt")
        print(f"Svaed ping result for {bot.get_name()}")
    except Exception as e:
        print(f"Error saving ping result from {bot.get_name()}: {e}")

#download_file command to get output files from bot nodes to local machine

# Run User pings for duration of whole experiment
print("\n===== Starting Baseline Testing =====")
user_thread = threading.Thread(target=start_user_ping, args=(user, ip_target))
user_thread.start()

time.sleep(3)

# Start DDoS Attack Simulation
print("\n===== Starting DDoS Simulation =====")

# Launch threads
threads = []
for bot in bots:
    thread = threading.Thread(target=start_ping_flood, args=(bot, ip_target))
    thread.start()
    threads.append(thread)

# Wait for all threads to start (not needed to wait for them to finish)
for thread in threads:
    thread.join()
    
# Collect statistics

#print("\n===== Monitoring Effects During Attack =====")
#print("\n--- Latency & Packet Loss (Target -> Controller) ---")
#print(target.execute(f"ping -i 0.5 -c 20 {controller.get_management_ip()}"))

#print("\n--- Latency & Packet Loss (User -> Target) ---")


#print("\n--- CPU Usage on Target ---")
#print(target.execute("top -bn1 | head -n 10"))

#print("\n--- Network Traffic Stats on Target ---")
#print(target.execute("ifconfig enp7s0"))

# Wait for attack and user to end
print("\n===== Wait for DDoS Attack and User Pings to End =====")

for bot in bots:
    wait_for_ping_to_finish(bot)

wait_for_ping_to_finish_user(user)

# Collect Results
print("\n===== Collecting Results =====")

print("\n--- Queue Status on Target ---")
print(target.execute("sudo tc -s qdisc show dev enp7s0"))

for bot in bots:
    collect_ping_results(bot)

collect_ping_results(user)

print("\nRestoring normal network conditions.")
target.execute("sudo tc qdisc del dev enp7s0 root") # Remove queue limit
print("\nDDoS simulation complete. Network restored.")

    


===== Starting Baseline Testing =====
Starting ping from User...

===== Starting DDoS Simulation =====
Starting ping flood attack from Bot_0...
Starting ping flood attack from Bot_2...
Starting ping flood attack from Bot_1...
Starting ping flood attack from Bot_3...

===== Wait for DDoS Attack and User Pings to End =====
Waiting for Bot_0 to finish ping flood...
Bot_0 finished.
Waiting for Bot_1 to finish ping flood...
Bot_1 finished.
Waiting for Bot_2 to finish ping flood...
Bot_2 finished.
Waiting for Bot_3 to finish ping flood...
Bot_3 finished.
Waiting for User to finish ping flood...
User finished ping.

===== Collecting Results =====

--- Queue Status on Target ---
qdisc tbf 8016: root refcnt 9 rate 1Kbit burst 10Kb limit 1Kb 
 Sent 16884 bytes 186 pkt (dropped 3994, overlimits 4241 requeues 0) 
 backlog 812b 10p requeues 0
('qdisc tbf 8016: root refcnt 9 rate 1Kbit burst 10Kb limit 1Kb \n Sent 16884 bytes 186 pkt (dropped 3994, overlimits 4241 requeues 0) \n backlog 812b 10p re

### 3. Cleanup Resources

In [34]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")